In [3]:
import os
from os.path import realpath, join

import drjit as dr
import mitsuba as mi

mi.set_variant('cuda_ad_rgb')
mi.set_log_level(mi.LogLevel.Warn)

In [7]:
config = {
    'render_resolution': (128, 128),
    'heightmap_resoulution': (512, 512),
    'n_upsampling_steps': 4,
    'spp': 32,
    'max_iterations': 1000,
    'learning_rate': 3e-5,
    'reference': 'scenes/tutorial4/references/sunday-512.exr'
}

In [8]:
image_ref = mi.TensorXf(mi.Bitmap(config['reference']))
print('[i] Reference image selected:', config['reference'])
mi.Bitmap(image_ref)

[i] Reference image selected: scenes/tutorial4/references/sunday-512.exr


Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [512, 512],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 3 MiB of image data ]
]

### Creating the emitter

In [11]:
bayer = dr.zeros(mi.TensorXf, (32, 32, 3))
bayer[ ::2, ::2,  2] = 2.2
bayer[ ::2, 1::2, 1] = 2.2
bayer[1::2, 1::2, 0] = 2.2

emitter = {
    'type': 'directionalarea',
    'radiance': {
        'type': 'bitmap',
        'bitmap': mi.Bitmap(bayer),
        'raw': True,
        'filter_type': 'nearest'
    },
}
mi.Bitmap(bayer)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [32, 32],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 12 KiB of image data ]
]

### Creating the integrator

In [12]:
integrator = {
    'type': 'ptracer',
    'samples_per_pass': 256,
    'max_depth': 4,
    'hide_emitters': False
}